In [1]:
import numpy as np
from typing import NamedTuple
import random as rd
from collections import defaultdict

In [2]:
with open('data/qualification_round_2019/d_pet_pictures.txt') as file:
    lines = file.read().splitlines()

NUMPHOTOS = int(lines[0])
RAW_PHOTOS = [line.split() for line in lines[1:]]

class Photo(NamedTuple):
    type: str
    len: int
    tags: tuple
    num: int

photos = [Photo(photo[0], int(photo[1]), tuple(photo[2:]),num) for num, photo in enumerate(RAW_PHOTOS)]

In [3]:
photos_H = sorted([photo for photo in photos if photo.type == 'H'])
photos_V = sorted([photo for photo in photos if photo.type == 'V'])
print(f'Horizontal: {len(photos_H)}\nVertical: {len(photos_V)}')

Horizontal: 30000
Vertical: 60000


In [4]:
def interest(tags):
    tags1 = set(tags[0])
    tags2 = set(tags[1])
    S1 = len(tags1 - tags2)
    S2 = len(tags1 & tags2)
    S3 = len(tags2 - tags1)
    return min(S1,S2,S3)

In [5]:
def random_partition(lst, parts = 2):
    percentage = 1/parts
    cut = int(len(lst)*percentage)+1
    if cut == 0:
        raise ValueError('Too Many Parts')
    rd.shuffle(lst)
    return [lst[i*cut:(i+1)*cut] for i in range(parts)]
    

In [6]:
tag_photo = defaultdict(list)
for photo in photos_H:
    for k in photo.tags:
        tag_photo[k].append(photo)

In [7]:
splits = random_partition(photos_H, int(NUMPHOTOS**0.5))
test = splits[0]

In [21]:
import time
def recursion():
    seen = set()
    chain = []
    pic_set = photos_H
    pic = rd.sample(photos_H,1)[0]
    counter = 0
    while pic_set and counter <= 10000:
        print('Counter: ', counter, end='\r')
        pic = max(rd.sample(pic_set,20), key=lambda x: interest((pic.tags,x.tags)))
        if not pic in seen:
            counter += 1
            chain.append(pic)
            seen.add(pic)
            pic_set = set(x for tag in pic.tags for x in tag_photo[tag])
            if pic_set.issubset(seen):
                break
        else:
            pic_set.remove(pic)
    return chain
t = time.time()
chain = recursion()
t2 = time.time()

Counter:  10000

In [22]:
score = sum([interest((x.tags, y.tags)) for x, y in zip(chain, chain[1:])])
print(score, len(chain), (t2 - t))

43290 10001 620.4208018779755
